# nf_loto_webui システム徹底チュートリアル

このノートブックでは、以下の流れで **nf_loto_webui** システムを一通り確認できるようにします。

1. プロジェクト構成の確認
2. PostgreSQL への接続確認
3. `nf_model_runs` テーブルの作成と確認
4. `nf_loto%` テーブルからのデータ取得確認
5. `run_loto_experiment` を使った単発実験の実行
6. `sweep_loto_experiments` を使った限定的なグリッド実験
7. `nf_model_runs` を使った実行履歴・メタ情報の確認
8. かんたんな統計的分析（モデル・backend ごとの比較など）

※ 実際の DB に `nf_loto%` テーブルが存在することを前提としています。

In [ ]:
# 1. プロジェクト構成の確認
import os, pathlib

root = pathlib.Path.cwd()
print("CWD:", root)
print("存在するファイル/ディレクトリ:")
for p in sorted(root.iterdir()):
    print("-", p)

print("\nsrc 以下:")
src = root / "src"
if src.exists():
    for p in sorted(src.rglob("*.py")):
        print("-", p.relative_to(root))
else:
    print("src ディレクトリが見つかりません。CWD が nf_loto_webui 直下になっているか確認してください。")

## 2. PostgreSQL 接続確認

`config/db_config.py` に定義された接続情報で DB に接続できるかを確認します。

In [ ]:
import psycopg2
from config.db_config import DB_CONFIG

print("DB_CONFIG:", DB_CONFIG)

try:
    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT version();")
            version = cur.fetchone()[0]
    print("PostgreSQL 接続成功:", version)
except Exception as e:
    print("PostgreSQL 接続に失敗しました。設定を確認してください。")
    raise e

## 3. `nf_model_runs` テーブルの作成と確認

実行ログ・リソース情報・パラメータを格納するテーブル `nf_model_runs` を作成し、構造を確認します。

In [ ]:
from pathlib import Path
import pandas as pd

sql_path = Path("sql/001_create_nf_model_run_tables.sql")
print("SQL ファイル:", sql_path, "exists=", sql_path.exists())

sql_text = sql_path.read_text(encoding="utf-8")

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        cur.execute(sql_text)
    conn.commit()

print("nf_model_runs テーブル作成 or 既存確認が完了しました。")

# テーブル一覧に nf_model_runs が存在するか確認
with psycopg2.connect(**DB_CONFIG) as conn:
    df_tables = pd.read_sql(
        """
        SELECT tablename
        FROM pg_catalog.pg_tables
        WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
          AND tablename = 'nf_model_runs'
        """,
        conn,
    )

df_tables

## 4. `nf_loto%` テーブルからのデータ取得確認

`src/data_access/loto_repository.py` を使って:

- `nf_loto%` テーブル一覧
- 任意テーブルの `loto` 一覧
- 任意 `loto` の `unique_id` 一覧
- パネルデータの先頭数行

を確認します。

In [ ]:
from src.data_access.loto_repository import (
    list_loto_tables,
    list_loto_values,
    list_unique_ids,
    load_panel_by_loto,
)

tables_df = list_loto_tables()
display(tables_df)

if not tables_df.empty:
    table_name = tables_df["tablename"].iloc[0]
    print("Using table:", table_name)
    loto_df = list_loto_values(table_name)
    display(loto_df)
    if not loto_df.empty:
        loto = loto_df["loto"].iloc[0]
        uid_df = list_unique_ids(table_name, loto)
        display(uid_df)
        unique_ids = uid_df["unique_id"].head(3).tolist()
        print("Using loto:", loto)
        print("Using unique_ids:", unique_ids)

        panel_df = load_panel_by_loto(table_name, loto, unique_ids)
        display(panel_df.head())
    else:
        print("loto が見つかりません。")
else:
    print("nf_loto% テーブルが存在しません。")

## 5. `run_loto_experiment` を使った単発実験

最も単純な設定で `run_loto_experiment` を 1 回実行し、

- 予測結果 `preds`
- メタ情報 `meta`
- `nf_model_runs` テーブルにログが残っているか

を確認します。

In [ ]:
from src.ml.model_runner import run_loto_experiment

if not tables_df.empty and not loto_df.empty and not uid_df.empty:
    preds, meta = run_loto_experiment(
        table_name=table_name,
        loto=loto,
        unique_ids=unique_ids,
        model_name="AutoTFT",
        backend="optuna",   # または "ray"
        horizon=28,
        loss="mse",
        metric="val_loss",
        num_samples=4,
    )

    print("run_id:", meta["run_id"])
    display(preds.head())
    print("meta:")
    display(meta)
else:
    print("前セルで nf_loto% テーブルの確認に失敗しているため、ここはスキップされます。")

## 6. `sweep_loto_experiments` による限定グリッド実験

今度は `sweep_loto_experiments` を使って、

- AutoModel: `AutoTFT`, `AutoNHITS`
- backend: `optuna`
- loss: `mse`, `mae`
- h: `14`, `28`
- early_stop: 有効/無効

など、小さめのグリッドを実行します。

In [ ]:
from src.ml.model_runner import sweep_loto_experiments
from src.ml.model_registry import list_automodel_names

if not tables_df.empty and not loto_df.empty and not uid_df.empty:
    model_names = [m for m in list_automodel_names() if m in ("AutoTFT", "AutoNHITS")]
    backends = ["optuna"]

    param_spec = {
        "loss": ["mse", "mae"],
        "h": [14, 28],
        "freq": ["D"],
        "local_scaler_type": ["robust"],
        "val_size": [28],
        "refit_with_val": [True],
        "use_init_models": [False],
        "early_stop": [True, False],
        "early_stop_patience_steps": [3],
    }

    results = sweep_loto_experiments(
        table_name=table_name,
        loto=loto,
        unique_ids=unique_ids,
        model_names=model_names,
        backends=backends,
        param_spec=param_spec,
        mode="grid",
        num_samples=2,
    )

    print("実行した本数:", len(results))
    if results:
        display(results[0].preds.head())
        display(results[0].meta)
else:
    print("nf_loto% テーブルが無いため、ここはスキップされます。")

## 7. `nf_model_runs` の中身を確認

ここまでに実行した実験が `nf_model_runs` に記録されているか確認し、

- モデル名
- backend
- horizon, loss
- status, duration_seconds

などをざっと眺めます。

In [ ]:
with psycopg2.connect(**DB_CONFIG) as conn:
    df_runs = pd.read_sql(
        "SELECT * FROM nf_model_runs ORDER BY id DESC LIMIT 100",
        conn,
    )

print("nf_model_runs 行数:", len(df_runs))
display(df_runs.head())

## 8. かんたんな分析例

ここでは `nf_model_runs` を使って、

- モデルごとの実行時間分布
- backend ごとの実行数

など、簡単な集計・分析を行います。より高度な統計的検定や因果推論などは、
この DataFrame を元に好みのライブラリで拡張してください。

In [ ]:
if not df_runs.empty:
    # backend ごとの件数
    print("backend ごとの件数:")
    display(df_runs["backend"].value_counts())

    # model_name ごとの件数
    if "model_name" in df_runs.columns:
        print("\nmodel_name ごとの件数:")
        display(df_runs["model_name"].value_counts())

    # duration_seconds の概要
    if "duration_seconds" in df_runs.columns:
        print("\nduration_seconds の統計量:")
        display(df_runs["duration_seconds"].describe())
else:
    print("nf_model_runs が空です。実験を実行した後に再度このセルを実行してください。")